In [ ]:
# Célula 0: Instalação de Dependências
print("📦 INSTALANDO DEPENDÊNCIAS...")
print("="*60)

# Dependências básicas
!pip install -q pandas numpy scikit-learn

# Sentence Transformers e PyTorch
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install -q sentence-transformers

# Transformers (para evitar erro de tf_keras)
!pip install -q transformers

# Visualização
!pip install -q matplotlib seaborn

print("\n✅ Todas as dependências instaladas!")

📦 INSTALANDO DEPENDÊNCIAS...
✅ Todas as dependências instaladas!


In [11]:
# Célula 1: Imports
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    classification_report, 
    accuracy_score, 
    roc_auc_score,
    confusion_matrix,
    roc_curve
)
from sklearn.preprocessing import LabelEncoder

# Sentence Transformers
from sentence_transformers import SentenceTransformer

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Todos os imports realizados com sucesso!")
print(f"📊 Pandas versão: {pd.__version__}")
print(f"🔢 NumPy versão: {np.__version__}")

ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [5]:
import pandas as pd
import numpy as np

# Carregar o dataset
print("📂 Carregando dataset...")
df = pd.read_csv("data_bq.csv")

# Informações básicas do dataset
print(f"\n✅ Dataset carregado com sucesso!")
print(f"📊 Shape: {df.shape}")
print(f"📋 Colunas: {list(df.columns)}")
print(f"\n🔍 Primeiras linhas:")
display(df.head())


📂 Carregando dataset...

✅ Dataset carregado com sucesso!
📊 Shape: (172758, 13)
📋 Colunas: ['codigo_item', 'descricao_item', 'codigo_glosa', 'descricao_glosa', 'valor_glosa', 'justificativa', 'data_realizacao', 'valor_cobrado', 'quantidade_item', 'retorno_operadora', 'pago', 'convenio', 'prestador']

🔍 Primeiras linhas:


,codigo_item,descricao_item,codigo_glosa,descricao_glosa,valor_glosa,justificativa,data_realizacao,valor_cobrado,quantidade_item,retorno_operadora,pago,convenio,prestador
0,40316122,Antígeno Carcinoembriogênico (Cea),1601,REINCIDENCIA NO ATENDIMENTO,140.60,40316122- cobrança realizada conforme senha au...,2024-06-08 03:00:00 UTC,140.60,1,REINCIDENCIA NO ATENDIMENTO,True,UnimedCNU,BP
1,78241030,Conector Spiros Macho S/Trava,1707,NAO EXISTE INFORMACAO SOBRE A TABELA QUE SERA ...,212.22,"78241030- conforme aditivo vigente , simpro pu...",2024-06-04 03:00:00 UTC,212.22,1,NAO EXISTE INFORMACAO SOBRE A TABELA QUE SERA ...,True,UnimedCNU,BP
2,78241057,Tampa esteril CH-CAP 10,1707,NAO EXISTE INFORMACAO SOBRE A TABELA QUE SERA ...,56.13,Considerar 78241057- conforme aditivo vigente ...,2024-06-04 03:00:00 UTC,56.13,1,NAO EXISTE INFORMACAO SOBRE A TABELA QUE SERA ...,True,UnimedCNU,BP
3,78241065,Conector Clave Spike P/Bolsa,1707,NAO EXISTE INFORMACAO SOBRE A TABELA QUE SERA ...,213.91,"78241065- conforme aditivo vigente , simpro pu...",2024-06-04 03:00:00 UTC,213.91,1,NAO EXISTE INFORMACAO SOBRE A TABELA QUE SERA ...,False,UnimedCNU,BP
4,78241090,Conector Valvulado Microclave,1705,VALOR APRESENTADO A MAIOR,67.92,"78241090- conforme aditivo vigente , simpro pu...",2024-06-04 03:00:00 UTC,84.30,1,VALOR APRESENTADO A MAIOR,False,UnimedCNU,BP


In [6]:
# Célula 2: Análise Exploratória das Justificativas
print("📝 ANÁLISE DAS JUSTIFICATIVAS")
print("="*60)

# Verificar justificativas
print(f"Total de justificativas: {len(df)}")
print(f"Justificativas nulas: {df['justificativa'].isnull().sum()}")
print(f"Justificativas vazias: {(df['justificativa'] == '').sum()}")

# Estatísticas de tamanho
df['just_length'] = df['justificativa'].fillna('').str.len()
df['just_words'] = df['justificativa'].fillna('').str.split().str.len()

print(f"\n📏 Tamanho das justificativas:")
print(f"   Média de caracteres: {df['just_length'].mean():.0f}")
print(f"   Mediana de caracteres: {df['just_length'].median():.0f}")
print(f"   Média de palavras: {df['just_words'].mean():.0f}")

print(f"\n🔍 Exemplos de justificativas:")
for i in range(min(3, len(df))):
    print(f"\n{i+1}. PAGO={df.iloc[i]['pago']}")
    print(f"   {df.iloc[i]['justificativa'][:150]}...")

# Análise por resultado
print(f"\n📊 Comparação PAGO vs NÃO PAGO:")
pago_df = df[df['pago'] == True]
nao_pago_df = df[df['pago'] == False]

print(f"   Média chars (PAGO): {pago_df['just_length'].mean():.0f}")
print(f"   Média chars (NÃO PAGO): {nao_pago_df['just_length'].mean():.0f}")
print(f"   Média palavras (PAGO): {pago_df['just_words'].mean():.0f}")
print(f"   Média palavras (NÃO PAGO): {nao_pago_df['just_words'].mean():.0f}")

📝 ANÁLISE DAS JUSTIFICATIVAS
Total de justificativas: 172758
Justificativas nulas: 0
Justificativas vazias: 0

📏 Tamanho das justificativas:
   Média de caracteres: 132
   Mediana de caracteres: 132
   Média de palavras: 18

🔍 Exemplos de justificativas:

1. PAGO=True
   40316122- cobrança realizada conforme senha autorizada, conta auditada e validada. Reprocessar....

2. PAGO=True
   78241030- conforme aditivo vigente , simpro pura cód 151030 vigente 11.03.24 R$ 212,22 cada. Reprocessar....

3. PAGO=True
   Considerar 78241057- conforme aditivo vigente , simpro pura tiss 151042 vigente 15.03.22 R$ 56,13  cada. Reprocessar....

📊 Comparação PAGO vs NÃO PAGO:
   Média chars (PAGO): 127
   Média chars (NÃO PAGO): 136
   Média palavras (PAGO): 18
   Média palavras (NÃO PAGO): 19


In [8]:
# Célula 3: Preparar texto para embeddings (SEM retorno_operadora)
from sklearn.model_selection import train_test_split

print("🔧 PREPARANDO TEXTOS PARA EMBEDDINGS")
print("="*60)

# ⚠️ IMPORTANTE: Usar APENAS a justificativa (sem retorno_operadora)
# Motivo: retorno_operadora é posterior à decisão = DATA LEAKAGE!

print("⚠️  ATENÇÃO: Usando APENAS 'justificativa' para evitar data leakage")
print("   'retorno_operadora' contém informação POSTERIOR à decisão de pagamento")

# Limpar e preparar justificativas
df['justificativa_clean'] = df['justificativa'].fillna('').astype(str).str.strip()

# Remover linhas com justificativas vazias
df_clean = df[df['justificativa_clean'] != ''].copy()

print(f"\n✅ Amostras após limpeza: {len(df_clean)}")
print(f"✅ Amostras removidas (vazias): {len(df) - len(df_clean)}")

# ✅ FEATURES: Apenas justificativa (dados disponíveis ANTES da decisão)
X_text = df_clean['justificativa_clean'].values

# ✅ TARGET: O que queremos prever
y = df_clean['pago'].values

# Converter y para binário (0 e 1)
y = y.astype(int)

print(f"\n📊 Distribuição do target:")
print(f"   PAGO (1): {sum(y)} ({sum(y)/len(y)*100:.1f}%)")
print(f"   NÃO PAGO (0): {len(y)-sum(y)} ({(len(y)-sum(y))/len(y)*100:.1f}%)")

# Split train/test
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

print(f"\n✅ Split realizado:")
print(f"   Treino: {len(X_train_text)} amostras")
print(f"   Teste: {len(X_test_text)} amostras")

print(f"\n🎯 Features usadas: APENAS justificativa (sem data leakage)")

🔧 PREPARANDO TEXTOS PARA EMBEDDINGS
⚠️  ATENÇÃO: Usando APENAS 'justificativa' para evitar data leakage
   'retorno_operadora' contém informação POSTERIOR à decisão de pagamento

✅ Amostras após limpeza: 172758
✅ Amostras removidas (vazias): 0

📊 Distribuição do target:
   PAGO (1): 71446 (41.4%)
   NÃO PAGO (0): 101312 (58.6%)

✅ Split realizado:
   Treino: 138206 amostras
   Teste: 34552 amostras

🎯 Features usadas: APENAS justificativa (sem data leakage)


In [9]:
# Célula 4: Criar embeddings com Sentence Transformers
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer
import time

print("🧠 GERANDO EMBEDDINGS SEMÂNTICOS")
print("="*60)

# Carregar modelo pré-treinado
# Opções:
# - 'all-MiniLM-L6-v2': Rápido, bom para português (384 dims)
# - 'paraphrase-multilingual-MiniLM-L12-v2': Melhor para PT-BR (384 dims)
# - 'distiluse-base-multilingual-cased-v2': Multilíngue robusto (512 dims)

model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
print(f"📥 Carregando modelo: {model_name}")

model = SentenceTransformer(model_name)

print(f"✅ Modelo carregado!")
print(f"📐 Dimensão dos embeddings: {model.get_sentence_embedding_dimension()}")

# Gerar embeddings para treino
print(f"\n🔄 Gerando embeddings de TREINO...")
start = time.time()
X_train_embeddings = model.encode(
    X_train_text,
    show_progress_bar=True,
    batch_size=32,
    convert_to_numpy=True
)
train_time = time.time() - start
print(f"✅ Embeddings de treino gerados em {train_time:.1f}s")
print(f"   Shape: {X_train_embeddings.shape}")

# Gerar embeddings para teste
print(f"\n🔄 Gerando embeddings de TESTE...")
start = time.time()
X_test_embeddings = model.encode(
    X_test_text,
    show_progress_bar=True,
    batch_size=32,
    convert_to_numpy=True
)
test_time = time.time() - start
print(f"✅ Embeddings de teste gerados em {test_time:.1f}s")
print(f"   Shape: {X_test_embeddings.shape}")

print(f"\n🎯 EMBEDDINGS PRONTOS PARA TREINAMENTO!")

C:\Users\bruno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\bruno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\bruno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode versi

ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
# Célula 5: Treinar modelo simples com embeddings
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

print("🌲 TREINANDO RANDOM FOREST COM EMBEDDINGS")
print("="*60)

# Treinar modelo
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)

print("🔄 Treinando...")
rf_model.fit(X_train_embeddings, y_train)

# Predições
y_pred = rf_model.predict(X_test_embeddings)
y_proba = rf_model.predict_proba(X_test_embeddings)[:, 1]

# Métricas
print(f"\n📊 RESULTADOS:")
print(f"✅ Acurácia: {accuracy_score(y_test, y_pred):.2%}")
print(f"✅ AUC-ROC: {roc_auc_score(y_test, y_proba):.2%}")

print(f"\n📋 Relatório Detalhado:")
print(classification_report(y_test, y_pred, target_names=['Não Pago', 'Pago']))

print(f"\n🎯 Modelo treinado com embeddings semânticos!")